In [1]:
from transformers import pipeline

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [3]:
model = AutoModelForCausalLM.from_pretrained('../hf-models/usvsnsp-sft-combined').half().eval()

In [4]:
tokenizer = AutoTokenizer.from_pretrained(
    'EleutherAI/pythia-12b', 
    cache_dir = './hf-models',
    additional_special_tokens = (
        '<|query|>',
        '</query/>',
        '<|response|>',
        '</response/>',
        '<|result|>',
        '</result/>',
        '<|summary|>',
        '</summary/>',
        '<|results|>',
        '</results/>',
        '<|result-sentence|>'
    )
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
pipe = pipeline('text-generation', model = model, tokenizer = tokenizer, device = 0)

In [8]:
pipe("<|prompt|>Hey I am a farmer from Hyderabad and am not able to figure out why my mango leaves turned yellow</prompt/><|summary|>", max_length=128)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


[{'generated_text': '<|prompt|>Hey I am a farmer from Hyderabad and am not able to figure out why my mango leaves turned yellow</prompt/><|summary|>.\n<prompt/>\n<prompt/>\n<prompt/>\n<prompt/>\n<prompt/>\n<prompt/>\n<prompt/>\n<prompt/>\n<prompt/>\n<prompt/>\n<prompt/>\n<prompt/>\n<prompt/>\n<prompt/>\n<prompt/>\n<prompt/>\n<prompt/>\n<prompt/>\n<prompt/>\n<prompt/>\n<prompt/>\n<prompt/>\n<prompt/>\n<prompt/>'}]